# Cleaning COVID-19 Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

%matplotlib inline

In [3]:
!pwd

/Users/ramonperez/Desktop/eda_covid19/code/clean


In [4]:
df = pd.read_csv('../../data/raw_data/owid-covid-data.csv')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
0,AFG,Asia,Afghanistan,2019-12-31,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
1,AFG,Asia,Afghanistan,2020-01-01,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
2,AFG,Asia,Afghanistan,2020-01-02,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
3,AFG,Asia,Afghanistan,2020-01-03,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
4,AFG,Asia,Afghanistan,2020-01-04,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5


In [ ]:
# filter out and select columns with less than 5% of missing values
# the index attribute allows you to select the name of the columns
to_drop = (missing_pct[missing_pct < 5]).index
to_drop

In [ ]:
# drop the subset of columns selected in the previous cell

df.dropna(subset=to_drop, inplace=True)
df.head()

In [ ]:
# check what else is missing

missing = ((df.isna().sum() / df.shape[0]) * 100)
missing

In [ ]:
# select any column with missing values

missing_cols = (missing[missing > 0]).index
len(missing_cols)

In [ ]:
# print columns, data types and the persentage of missing
# values of such columns

for col in missing_cols: # iterate over columns with missing values
    print(f"Column --> {col} --> {df[col].dtype} --> {df[col].isna().sum()/df.shape[0]*100:.2f}!")

In [ ]:
# select columns with more than 50% missing values

missing_more_50 = (missing[missing > 50]).index
len(missing_more_50)

In [ ]:
# drop columns with more than 50% missing values

df.drop(labels=missing_more_50, axis=1, inplace=True)

In [ ]:
missing_less_15 = (missing[(missing > 0) & (missing <= 15)]).index
len(missing_less_15)

In [ ]:
missing_15 = missing_less_15.drop('gdp_per_capita')

In [ ]:
for col in missing_15:
    df[col].fillna(value=df[col].median(), axis=0, inplace=True)
    
df.isna().sum()

In [ ]:
for col in ['female_smokers', 'male_smokers']:
    df[col].fillna(value=df[col].median(), axis=0, inplace=True)

In [ ]:
df['extreme_poverty'].fillna(df['extreme_poverty'].median(), inplace=True)

In [ ]:
df['stringency_index'].fillna(value=0, inplace=True)

In [ ]:
countries_missing = df.loc[df['gdp_per_capita'].isna(), 'location'].unique()
countries_missing

In [ ]:
df.dropna(how='any', axis=0, inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['week'] = df['date'].dt.week
df['weekday'] = df['date'].dt.weekday
df['quarter'] = df['date'].dt.quarter
df['day_of_week'] = df['date'].dt.day_name()
df['week_or_end'] = df['weekday'].apply(lambda x: 'weekend' if x >= 5 else 'week_day')

In [ ]:
df.to_csv('covid19_ready_data.csv', index=False)